In [1]:
import pandas as pd
file_path = 'All_Data_Cleaned.csv'
data =pd.read_csv(file_path)
data2=pd.read_csv('Arabic_Data_cleaned_without_duplicated.csv')
data

,Arabic_Tweets_Cleaned,labels_new
0,اضافه قيام معلمو جيزه ذهاب جريد سابع نشر خبر ا...,MSA
1,بعد الله حرام تجي تلقي واقف اشاره عرض متعدي نا...,MSA
2,لمس يد مرة واضح جد حكم,DIAL_LEV
3,خصوص هاتريك عمر راح صار,DIAL_LEV
4,الله أجبر كسر رجع أدهم قبل بكر,DIAL_GLF
...,...,...
85278,بورسعيد حكاية,DIAL_LEV
85279,نعم لبس زي رسمي سعودي نظر علي اعين,MSA
85280,اتوقع الله اعلم قصد بقي شعب مسافر شل عد لا كلام,DIAL_GLF
85281,يتبدا,DIAL_EGY


In [2]:
#text = data['Arabic_Tweets_Cleaned'].values.tolist()
#target = data['labels_new'].values.tolist()

#text2 = data2['Arabic_Tweets_Cleaned'].values.tolist()
#target2 = data2['labels_new'].values.tolist()
text = data['Arabic_Tweets_Cleaned'].to_numpy()
target = data['labels_new'].to_numpy()


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text,target,test_size=0.2, shuffle = True,  random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(text2,target2,test_size=0.2, shuffle = True,  random_state=42)


In [3]:
import gensim

from gensim.models import Word2Vec
model_w2v = gensim.models.Word2Vec(text, size=100, window=5, min_count=10, workers=8)


In [4]:
weight_w2v = model_w2v.wv.vectors
vocab_size, embedding_size = weight_w2v.shape
print("Vocabulary Size: {} - Embedding Dim: {}".format(vocab_size, embedding_size))



Vocabulary Size: 88 - Embedding Dim: 100


In [5]:
import numpy as np
embedding_matrix = np.zeros((len(model_w2v.wv.vocab) + 1, 100))
for i,vec in enumerate(model_w2v.wv.vectors):
    embedding_matrix[i] = vec

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text,target,test_size=0.2, shuffle = True,  random_state=42)


In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

model = Sequential()
model.add(Embedding(len(model_w2v.wv.vocab)+1,100,input_length=text.shape[0] ,weights=[embedding_matrix],trainable=False))


In [16]:
text.shape

(85283,)

In [17]:
target.shape

(85283,)

In [8]:
model.add(LSTM(100,return_sequences=True))
model.add(Dense(target.shape[0],activation="softmax"))
model.summary()
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 85283, 100)        8900      
_________________________________________________________________
lstm (LSTM)                  (None, 85283, 100)        80400     
_________________________________________________________________
dense (Dense)                (None, 85283, 85283)      8613583   
Total params: 8,702,883
Trainable params: 8,693,983
Non-trainable params: 8,900
_________________________________________________________________


In [10]:
from tensorflow.keras.utils import to_categorical


batch = 32
epochs = 12
#y_train = to_categorical(y_train, 4)
#y_test = to_categorical(y_test, 4)

model.fit(X_train,y_train,batch,epochs)
model.evaluate(X_test,y_test)


Epoch 1/12


ValueError: in user code:

    C:\Users\Salma\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\Salma\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Salma\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Salma\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Salma\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    C:\Users\Salma\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\compile_utils.py:205 __call__
